In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional
from keras.optimizers import Adam, AdamW
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
# Загружаем данные из датасета (предположим, что данные находятся в формате CSV)
import pandas as pd

data = pd.read_csv('ipa2020.csv', names=['word', 'transcription'])

# Подготовка данных для модели
# Преобразуем слова и транскрипции в последовательности чисел
char_to_index = {}
index_to_char = {}

def create_mapping(data):
    global char_to_index, index_to_char
    unique_chars = set(''.join(data['word']) + ''.join(data['transcription']))
    char_to_index = {char: idx + 1 for idx, char in enumerate(unique_chars)}
    index_to_char = {idx + 1: char for idx, char in enumerate(unique_chars)}
    char_to_index['<PAD>'] = 0
    index_to_char[0] = '<PAD>'

create_mapping(data)

# Кодируем строки как последовательности индексов
max_word_length = max(data['word'].apply(len))
max_transcription_length = max(data['transcription'].apply(len))

def encode_sequence(sequence, max_length):
    return [char_to_index[char] for char in sequence] + [char_to_index['<PAD>']] * (max_length - len(sequence))

X = np.array([encode_sequence(word, max_word_length) for word in data['word']])
y = np.array([encode_sequence(transcription, max_transcription_length) for transcription in data['transcription']])

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем модель
model = Sequential([
    # Слой встраивания для входных символов
    Embedding(input_dim=len(char_to_index), output_dim=64),

    # Двунаправленный LSTM для обработки последовательности символов
    Bidirectional(LSTM(128, return_sequences=True)),

    # Полносвязный слой для прогнозирования каждого символа транскрипции
    TimeDistributed(Dense(len(char_to_index), activation='softmax'))
])

# Компилируем модель
model.compile(optimizer=AdamW(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Подготавливаем данные для обучения (добавляем размерности для y)
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

# Обучение модели
# Приведение длины транскрипций к длине входных слов
def encode_sequence(sequence, max_length):
    return [char_to_index[char] for char in sequence] + [char_to_index['<PAD>']] * (max_length - len(sequence))

# Приводим длины транскрипций к длине слов
max_length = max(max_word_length, max_transcription_length)
X = np.array([encode_sequence(word, max_length) for word in data['word']])
y = np.array([encode_sequence(transcription, max_length) for transcription in data['transcription']])

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Модель
model = Sequential([
    # Слой встраивания для входных символов
    Embedding(input_dim=len(char_to_index), output_dim=64),


    # Двунаправленный LSTM для обработки последовательности символов
    Bidirectional(LSTM(128, return_sequences=True)),

    # Полносвязный слой для прогнозирования каждого символа транскрипции
    TimeDistributed(Dense(len(char_to_index), activation='softmax'))
])

# Компиляция модели
model.compile(optimizer=AdamW(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Подготовка целевых данных (убираем лишнюю размерность)
y_train = np.expand_dims(y_train, -1)
y_test = np.expand_dims(y_test, -1)

# Обучение модели
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=20)


# Функция для предсказания
def predict_transcription(word):
    encoded_word = encode_sequence(word, max_length)
    encoded_word = np.expand_dims(encoded_word, axis=0)

    predicted_indices = model.predict(encoded_word)
    predicted_indices = np.argmax(predicted_indices, axis=-1)[0]

    transcription = ''.join(index_to_char[idx] for idx in predicted_indices if idx != 0)
    return transcription

# Пример
example_word = "пример"
print(f"{example_word} - {predict_transcription(example_word)}")

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.4641 - loss: 3.3738 - val_accuracy: 0.5824 - val_loss: 1.6300
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5939 - loss: 1.5794 - val_accuracy: 0.6078 - val_loss: 1.5130
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6036 - loss: 1.5208 - val_accuracy: 0.5951 - val_loss: 1.4555
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6090 - loss: 1.4454 - val_accuracy: 0.6049 - val_loss: 1.4185
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6139 - loss: 1.4411 - val_accuracy: 0.6115 - val_loss: 1.3873
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6023 - loss: 1.4268 - val_accuracy: 0.6148 - val_loss: 1.3562
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6136 - loss: 1.3502 - val_accuracy: 0.6225 - val_loss: 1.3135
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6310 - loss: 1.2964 - val_accuracy: 0.6422 - v

In [5]:
example_word = "пингвин"
print(f"{example_word} - {predict_transcription(example_word)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
пингвин - pʲɪˈˈnnnnj
